In [1]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import os 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
training_dataset_path = './Training Set'
test_dataset_path = './Test Set'

In [3]:
mean = [0.4243, 0.4522, 0.5050]
std = [0.2150, 0.2035, 0.1913]

training_transforms = transforms.Compose([
    transforms.Resize((224,224)), # Tweak this later on to determine best size
    transforms.RandomHorizontalFlip(), # Depends on the data, experiment here again
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])
test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [4]:
training_dataset = torchvision.datasets.ImageFolder(root = training_dataset_path, transform = training_transforms)
test_dataset = torchvision.datasets.ImageFolder(root = test_dataset_path, transform = test_transforms)

In [5]:
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True) # shuffled for better generalization
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False) # Not shuffled since actual comparison data

In [6]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [7]:
resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
num_classes = 6
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)
device = set_device()
resnet18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss() # Measures the error of the model

# SGD = Stochastic Gradient Descent (will apply Mini Gradient Descent)
# lr = learning rate (how much model will change in response to error)
# momentum (accelerates gradient vectors in the right direction)
# weight_decay (gives extra error for incorrect prediction, reduces overfitting)

optimizer = optim.SGD(resnet18_model.parameters(),lr=0.01, momentum=0.9, weight_decay=0.003)

In [8]:
def train_nn(model, training_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()
    best_accuracy = 0
    
    for epoch in range(n_epochs):
        print(f'Epoch number: {epoch+1}') # placeholder value syntax
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for batch in training_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device) # using CUDA if available instead of CPU
            total += labels.size(0) # size(0) contains batch size

            optimizer.zero_grad() # sets gradients to 0 (math stuff I dont understand)
            outputs = model(images) # model returns raw scores representing predictions
            _, predicted = torch.max(outputs.data, 1) # Extracts the most likely class label 
            loss = criterion(outputs, labels) # Compares outputs (what the model classified) and labels (actual value) with the CEL func
            loss.backward() # backpropagation
            optimizer.step() # update weights

            running_loss += loss.item() # Adds loss value, which is just a reference decimal to overall accuracy
            running_correct += (predicted==labels).sum().item() # Creates true/false tensor, and adds their binary values to give correct predictions count

        epoch_loss = running_loss/len(training_loader) # Divides running_loss over number of batches in epoch
        epoch_accuracy = 100 * running_correct/total
    
        print(f'   -Training: {running_correct} images predicted correctly out of {total} (Accuracy: {epoch_accuracy:.3f}%, Epoch Loss: {epoch_loss:.3f}%)') 
        
        test_accuracy = evaluate_model_on_test_set(model, test_loader)
        
        if(test_accuracy>best_accuracy):
            best_accuracy = test_accuracy
            save_checkpoint(model, epoch, optimizer, best_accuracy)
        
    print('Finished')
    return model

In [9]:
def evaluate_model_on_test_set(model, test_loader):
    model.eval() # sets model to evaluate mode
    epoch_correct_predictions = 0 # initiliazied counter
    total = 0 # initialized counter
    device = set_device() # CPU -> GPU
    
    with torch.no_grad(): # disables gradient descent, not allowing backpropagation (not needed here) and it saves memory and time
        for batch in test_loader:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            outputs = model(images) 
            _, predicted = torch.max(outputs.data, 1)
            epoch_correct_predictions += (predicted==labels).sum().item()

    epoch_accuracy = 100 * epoch_correct_predictions/total
    
    print(f'   -Testing: {epoch_correct_predictions} images predicted correctly out of {total} (Accuracy: {epoch_accuracy:.3f})%') 

    return epoch_accuracy

In [10]:
def save_checkpoint(model, epoch, optimizer, accuracy):
    # creates a state dictionary containing information of current model iteration
    state = {
        'epoch': epoch+1,
        'model' : model.state_dict(), # creates another state dictionary containing learned parameters of the model
        'accuracy' : accuracy,
        'optimizer': optimizer.state_dict() # creates another state dictionary containing optimizer information (lr,momentum, weight_decay)
    }
    torch.save(state, 'Best-Model-Checkpoint.pth.tar')
    

In [11]:
train_nn(resnet18_model, training_loader, test_loader, loss_fn, optimizer, 150)

Epoch number: 1
   -Training: 233.0 images predicted correctly out of 691 (Accuracy: 33.719%, Epoch Loss: 1.676%)
   -Testing: 69 images predicted correctly out of 194 (Accuracy: 35.567)%
Epoch number: 2
   -Training: 355.0 images predicted correctly out of 691 (Accuracy: 51.375%, Epoch Loss: 1.338%)
   -Testing: 90 images predicted correctly out of 194 (Accuracy: 46.392)%
Epoch number: 3
   -Training: 375.0 images predicted correctly out of 691 (Accuracy: 54.269%, Epoch Loss: 1.307%)
   -Testing: 60 images predicted correctly out of 194 (Accuracy: 30.928)%
Epoch number: 4
   -Training: 385.0 images predicted correctly out of 691 (Accuracy: 55.716%, Epoch Loss: 1.339%)
   -Testing: 101 images predicted correctly out of 194 (Accuracy: 52.062)%
Epoch number: 5
   -Training: 410.0 images predicted correctly out of 691 (Accuracy: 59.334%, Epoch Loss: 1.122%)
   -Testing: 89 images predicted correctly out of 194 (Accuracy: 45.876)%
Epoch number: 6
   -Training: 430.0 images predicted correc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
# Checks checkpoint information

checkpoint = torch.load('Best-Model-Checkpoint.pth.tar')

print(checkpoint['epoch'])
print(checkpoint['accuracy'])

146
77.31958762886597


In [13]:
# Save model with best accuracy

checkpoint = torch.load('Best-Model-Checkpoint.pth.tar')
resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
num_classes = 6
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)
resnet18_model.load_state_dict(checkpoint['model']) # optimizer not needed since it is only used during training to find best paramters

torch.save(resnet18_model, 'Best-Model.pth')